Cek jumlah beserta struktur datasets

In [4]:
import os

root_path = "D:\Pothole Vision - AI Road Damage Detection\dataset\RDD2022_all_countries"
def count_files(path):
    return len([f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]) if os.path.exists(path) else 0

summary = []

for country in os.listdir(root_path):
    country_path = os.path.join(root_path, country)
    if os.path.isdir(country_path):
        train_images = count_files(os.path.join(country_path, "train", "images"))
        test_images = count_files(os.path.join(country_path, "test", "images"))
        xml_files = count_files(os.path.join(country_path, "train", "annotations", "xmls"))
        summary.append(f"{country} -> Train Images: {train_images}, Test Images: {test_images}, Annotations: {xml_files}")

for line in summary:
    print(line)

China_Drone -> Train Images: 2401, Test Images: 0, Annotations: 2401
China_MotorBike -> Train Images: 1977, Test Images: 500, Annotations: 1977
Czech -> Train Images: 2829, Test Images: 709, Annotations: 2829
India -> Train Images: 7706, Test Images: 1959, Annotations: 7706
Japan -> Train Images: 10506, Test Images: 2627, Annotations: 10506
Norway -> Train Images: 8161, Test Images: 2040, Annotations: 8161
United_States -> Train Images: 4805, Test Images: 1200, Annotations: 4805


China_drone tidak punya folder test, abaikan?
Selanjutnya Saya akan split datset dari Train & Test menjadi Train, Test, & Val


In [ ]:
Jika dataset DIGABUNG semua negara menjadi satu dataset besar:
Keuntungan:

Model akan lebih general karena belajar dari berbagai jenis jalan, cuaca, kamera.

Bisa membantu jika nanti digunakan di Indonesia yang belum punya data.

Jumlah data menjadi sangat besar (10.000++), sangat bagus untuk deep learning.

Kekurangan:

Bisa menyebabkan bias ke negara dengan data terbanyak (misalnya India, Japan).

Anotasi antar negara mungkin memiliki inkonsistensi kecil (labeling style, noise).

In [ ]:
Rekomendasi untuk kasus ini:
Karena kamu akan pakai untuk Indonesia, tapi belum punya data lokal, maka:

Gabungkan semua negara → latih model global, supaya kuat terhadap variasi.

Simpan metadata negara asalnya → bisa dipakai untuk evaluasi per negara.

Nanti, jika ada data Indonesia, kamu bisa fine-tune model global ke data lokal.

In [ ]:
import os
import random
import shutil
import pandas as pd

# Lokasi dataset dan output
root_path = "D:\\Pothole Vision - AI Road Damage Detection\\dataset\\RDD2022_all_countries"
output_path = "D:\\Pothole Vision - AI Road Damage Detection\\dataset-mix"
train_val_split = 0.8  # 80% untuk train, 20% untuk val

# Membuat direktori output
os.makedirs(os.path.join(output_path, 'train', 'images'), exist_ok=True)
os.makedirs(os.path.join(output_path, 'train', 'annotations'), exist_ok=True)
os.makedirs(os.path.join(output_path, 'val', 'images'), exist_ok=True)
os.makedirs(os.path.join(output_path, 'val', 'annotations'), exist_ok=True)
os.makedirs(os.path.join(output_path, 'test', 'images'), exist_ok=True)

# Metadata untuk csv
metadata = []

def copy_file(src_file, dest_file):
    shutil.copy(src_file, dest_file)

# Proses per negara
for country in os.listdir(root_path):
    country_path = os.path.join(root_path, country)
    if not os.path.isdir(country_path):
        continue

    images_path = os.path.join(country_path, 'train', 'images')
    annotations_path = os.path.join(country_path, 'train', 'annotations', 'xmls')

    if not os.path.exists(images_path) or not os.path.exists(annotations_path):
        continue

    image_files = sorted([f for f in os.listdir(images_path) if f.endswith('.jpg')])
    annotation_files = sorted([f for f in os.listdir(annotations_path) if f.endswith('.xml')])

    for image, annotation in zip(image_files, annotation_files):
        # Gunakan nama file apa adanya
        src_image = os.path.join(images_path, image)
        src_annotation = os.path.join(annotations_path, annotation)

        split = 'train' if random.random() < train_val_split else 'val'
        dest_image = os.path.join(output_path, split, 'images', image)
        dest_annotation = os.path.join(output_path, split, 'annotations', annotation)

        copy_file(src_image, dest_image)
        copy_file(src_annotation, dest_annotation)

        metadata.append({'filename': image, 'country': country, 'split': split})

    # Test set
    test_images_path = os.path.join(country_path, 'test', 'images')
    if os.path.exists(test_images_path):
        for test_img in os.listdir(test_images_path):
            if test_img.endswith('.jpg'):
                src_test_img = os.path.join(test_images_path, test_img)
                dest_test_img = os.path.join(output_path, 'test', 'images', test_img)
                copy_file(src_test_img, dest_test_img)

# Simpan metadata
metadata_df = pd.DataFrame(metadata)
metadata_df.to_csv(os.path.join(output_path, 'metadata.csv'), index=False)

print("✅ Dataset berhasil diproses")

✅ Dataset berhasil digabung dan di-split.


In [41]:
import os
import pandas as pd
from tabulate import tabulate

root_path = "D:\\Pothole Vision - AI Road Damage Detection\\dataset-mix"

def count_files(path, ext):
    return len([f for f in os.listdir(path) if f.endswith(ext)]) if os.path.exists(path) else 0

# Data jumlah file berdasarkan split
data = {
    'Split': ['Train', 'Val', 'Test'],
    'Images': [
        count_files(os.path.join(root_path, 'train', 'images'), '.jpg'),
        count_files(os.path.join(root_path, 'val', 'images'), '.jpg'),
        count_files(os.path.join(root_path, 'test', 'images'), '.jpg')
    ],
    'XML Annotations': [
        count_files(os.path.join(root_path, 'train', 'annotations'), '.xml'),
        count_files(os.path.join(root_path, 'val', 'annotations'), '.xml'),
        '-'  # Test tidak memiliki anotasi
    ]
}

df_split = pd.DataFrame(data)

# Membaca metadata.csv
metadata_path = os.path.join(root_path, 'metadata.csv')
if os.path.exists(metadata_path):
    metadata = pd.read_csv(metadata_path)
    split_counts = metadata['split'].value_counts().reset_index()
    split_counts.columns = ['Split', 'Count']

    country_counts = metadata['country'].value_counts().reset_index()
    country_counts.columns = ['Country', 'Count']

    # Tampilkan tabel
    print("=== Jumlah File per Split ===")
    print(tabulate(df_split, headers='keys', tablefmt='pretty', showindex=False))
    print("\n=== Distribusi Metadata per Split ===")
    print(tabulate(split_counts, headers='keys', tablefmt='pretty', showindex=False))
    print("\n=== Distribusi Metadata per Negara ===")
    print(tabulate(country_counts, headers='keys', tablefmt='pretty', showindex=False))
else:
    print("File metadata.csv tidak ditemukan.")

=== Jumlah File per Split ===
+-------+--------+-----------------+
| Split | Images | XML Annotations |
+-------+--------+-----------------+
| Train | 30820  |      30820      |
|  Val  |  7565  |      7565       |
| Test  |  9035  |        -        |
+-------+--------+-----------------+

=== Distribusi Metadata per Split ===
+-------+-------+
| Split | Count |
+-------+-------+
| train | 30820 |
|  val  | 7565  |
+-------+-------+

=== Distribusi Metadata per Negara ===
+-----------------+-------+
|     Country     | Count |
+-----------------+-------+
|      Japan      | 10506 |
|     Norway      | 8161  |
|      India      | 7706  |
|  United_States  | 4805  |
|      Czech      | 2829  |
|   China_Drone   | 2401  |
| China_MotorBike | 1977  |
+-----------------+-------+


In [42]:
import os
import xml.etree.ElementTree as ET
from collections import Counter

root_path = "D:/Pothole Vision - AI Road Damage Detection/dataset-mix"
annotation_paths = [
    os.path.join(root_path, 'train', 'annotations'),
    os.path.join(root_path, 'val', 'annotations')
]

all_labels = []

# Loop seluruh file XML di train & val
for folder in annotation_paths:
    for filename in os.listdir(folder):
        if filename.endswith('.xml'):
            file_path = os.path.join(folder, filename)
            tree = ET.parse(file_path)
            root = tree.getroot()
            for obj in root.findall('object'):
                label = obj.find('name').text.strip()
                all_labels.append(label)

# Hitung semua kemunculan label
label_counts = Counter(all_labels)

print("Distribusi label di semua XML:")
for label, count in label_counts.items():
    print(f"{label}: {count}")

if all(label == "D00" for label in label_counts):
    print("\n✅ Semua file hanya mengandung label D00.")
else:
    print("\n⚠️ Ada label lain selain D00!")


Distribusi label di semua XML:
D10: 11830
D00: 26016
D20: 10617
Repair: 1046
D40: 6544
Block crack: 3
D44: 5057
D01: 179
D11: 45
D43: 793
D50: 3581
D0w0: 1

⚠️ Ada label lain selain D00!


In [43]:
import os
import xml.etree.ElementTree as ET

# Lokasi dataset
root_path = "D:/Pothole Vision - AI Road Damage Detection/dataset-mix"
annotation_dirs = [
    os.path.join(root_path, 'train', 'annotations'),
    os.path.join(root_path, 'val', 'annotations')
]
image_dirs = [
    os.path.join(root_path, 'train', 'images'),
    os.path.join(root_path, 'val', 'images')
]

# Inisialisasi counter
total_xml = 0
removed_count = 0

# Loop direktori anotasi dan gambar
for ann_dir, img_dir in zip(annotation_dirs, image_dirs):
    for filename in os.listdir(ann_dir):
        if not filename.endswith('.xml'):
            continue

        total_xml += 1
        xml_path = os.path.join(ann_dir, filename)
        tree = ET.parse(xml_path)
        root = tree.getroot()

        labels = [obj.find('name').text.strip() for obj in root.findall('object')]
        if not labels or any(label != "D00" for label in labels):
            # Hapus XML dan gambar
            os.remove(xml_path)
            image_file = filename.replace('.xml', '.jpg')
            image_path = os.path.join(img_dir, image_file)
            if os.path.exists(image_path):
                os.remove(image_path)
            removed_count += 1

total_xml, removed_count, total_xml - removed_count  # before, removed, after count

(38385, 31909, 6476)

In [44]:
import os

def count_files(path, ext):
    return len([f for f in os.listdir(path) if f.endswith(ext)]) if os.path.exists(path) else 0

base = "D:/Pothole Vision - AI Road Damage Detection/dataset-mix"

report = {
    "Split": [],
    "Images (.jpg)": [],
    "Annotations (.xml)": []
}

for split in ['train', 'val', 'test']:
    img_count = count_files(os.path.join(base, split, 'images'), '.jpg')
    xml_count = count_files(os.path.join(base, split, 'annotations'), '.xml') if split != 'test' else '-'
    
    report["Split"].append(split.capitalize())
    report["Images (.jpg)"].append(img_count)
    report["Annotations (.xml)"].append(xml_count)

import pandas as pd
from tabulate import tabulate

df = pd.DataFrame(report)
print(tabulate(df, headers="keys", tablefmt="pretty", showindex=False))

+-------+---------------+--------------------+
| Split | Images (.jpg) | Annotations (.xml) |
+-------+---------------+--------------------+
| Train |     5237      |        5237        |
|  Val  |     1239      |        1239        |
| Test  |     9035      |         -          |
+-------+---------------+--------------------+


In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.yaml')  # atau ganti ke 'yolov8s.yaml' jika mau versi sedikit lebih besar

model.train(
    data='rdd2022.yaml',
    epochs=20,
    imgsz=640,
    batch=16,
    name='yolov8_d00_only',
    project='runs/train'
)
